In [1]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.3 MB/s eta 0:00:00


In [2]:

import torch
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2Model
import re
import torch.nn as nn
import pandas as pd
import numpy as np
import csv
from google.colab import drive


import pickle
from konlpy.tag import Okt
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression

import textwrap


In [3]:
# 일단 자기 구글 드라이브에 학습된, 모델들 다 업로드 미리 해놔야 합니다.
# 이 노트는 학습된 모델들을 불러오고 사용하는 클래스들입니다.
# 만약 학습을 다시 해야 한다면 따로 학습시키고, 여기로 가져오고 ModelManager클래스에 통합 시켜서 돌려야해요.
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 요리명 기반 응답 모델
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

class BasedNameModel():
    def __init__(self):
        # 특수 토큰 정의
        self.Q_TKN = "<usr>"
        self.A_TKN = "<sys>"
        self.BOS = '</s>'
        self.EOS = '</s>'
        self.MASK = '<unused0>'
        self.SENT = '<unused1>'
        self.PAD = '<pad>'

        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2',
                                                                 bos_token=self.BOS,
                                                                 eos_token=self.EOS,
                                                                 unk_token="<unk>",
                                                                 pad_token=self.PAD,
                                                                 mask_token=self.MASK)
        self.model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        state_dict = torch.load('/content/drive/MyDrive/Final_project/based_name/kogpt2_model_3.pt', map_location=self.device)

        self.model.load_state_dict(state_dict)
        self.model.to(self.device)

    def predict(self, user_text):
        self.model.eval()
        with torch.no_grad():
          a = ''
          while True:
            input_ids = torch.LongTensor(self.tokenizer.encode(self.Q_TKN + user_text + self.SENT + self.A_TKN + a,
                                                               add_special_tokens=True)).unsqueeze(dim=0)
            input_ids = input_ids.to(self.device)
            outputs = self.model(input_ids)

            logits = outputs.logits

            pred = torch.argmax(logits, dim=-1)
            gen_token = self.tokenizer.convert_ids_to_tokens(pred[0].cpu().numpy().tolist())[-1]
            if gen_token == self.EOS:
              break
            a += gen_token.replace("▁", " ")

        return a.strip()


# # 사용 예시
# model = BasedNameModel()
# while True:
#   user_text = input('사용자: ')
#   if user_text == 'quit':
#     break
#   print('챗봇:', model.predict(user_text))


In [5]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

class BasedIngredientModel():
    def __init__(self):
        # 특수 토큰 정의
        self.Q_TKN = "<usr>"
        self.A_TKN = "<sys>"
        self.BOS = '</s>'
        self.EOS = '</s>'
        self.MASK = '<unused0>'
        self.SENT = '<unused1>'
        self.PAD = '<pad>'

        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2',
                                                                 bos_token=self.BOS,
                                                                 eos_token=self.EOS,
                                                                 unk_token="<unk>",
                                                                 pad_token=self.PAD,
                                                                 mask_token=self.MASK)
        self.model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        state_dict = torch.load('/content/drive/MyDrive/Final_project/based_ingredient/Ingredient_base_QA_model.pt', map_location=self.device)
        self.model.load_state_dict(state_dict)
        self.model.to(self.device)

    def predict(self, user_text):
        self.model.eval()
        with torch.no_grad():
          a = ''
          while True:
            input_ids = torch.LongTensor(self.tokenizer.encode(self.Q_TKN + user_text + self.SENT + self.A_TKN + a,
                                                               add_special_tokens=True)).unsqueeze(dim=0)
            input_ids = input_ids.to(self.device)
            outputs = self.model(input_ids)

            logits = outputs.logits

            pred = torch.argmax(logits, dim=-1)
            gen_token = self.tokenizer.convert_ids_to_tokens(pred[0].cpu().numpy().tolist())[-1]
            if gen_token == self.EOS:
              break
            a += gen_token.replace("▁", " ")

        return a.strip()

# 사용 예시
# model = BasedIngredientModel()
# while True:
#   user_text = input('사용자: ')
#   if user_text == 'quit':
#     break
#   print('챗봇:', model.predict(user_text))

In [6]:

# 주재료/부재료 분류 모델
# Okt 이용

class ClassifierIngredients():
    def __init__(self):
        self.okt = Okt()
        self.model_path = '/content/drive/MyDrive/Final_project/classifier_ingredient/trained_classifier.pkl'

        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')

        with open(self.model_path, 'rb') as file:
            self.classifier = pickle.load(file)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def predict(self, user_input):

        nouns = self.okt.nouns(user_input)
        classified_ingredients = {'기본 재료': [], '부재료': []}
        for noun in nouns:

            inputs = self.tokenizer(noun, return_tensors="pt", padding=True, truncation=True, max_length=512)
            inputs = inputs.to(self.device)

            with torch.no_grad():
                outputs = self.model(**inputs)
            noun_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

            # 분류
            label_pred = self.classifier.predict([noun_embedding])[0]

            # 결과에 따라 분류
            if label_pred == 0:
                classified_ingredients['기본 재료'].append(noun)
            elif label_pred == 1:
                classified_ingredients['부재료'].append(noun)


        return classified_ingredients


# model = ClassifierIngredients()
# result = model.predict('오징어, 돼지고기, 시금치, 설탕, 간장, 소금, 물, 불, 가스, 그릇')
# print(result)


In [7]:
# 질문 타입 분류기 모델 정의
class QuestionClassification(nn.Module):
    def __init__(self, gpt2_model_name='skt/kogpt2-base-v2', num_labels=4):
        super().__init__()
        self.gpt2 = GPT2Model.from_pretrained(gpt2_model_name)
        self.classifier = nn.Linear(self.gpt2.config.hidden_size, num_labels)
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained(
            gpt2_model_name,
            bos_token="</s>",
            eos_token="</s>",
            unk_token="<unk>",
            pad_token="<pad>",
            mask_token="<unused0>"
        )
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, input_ids, attention_mask):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(cls_token)
        return logits

    def clean_text(self, text):
        return re.sub(r"([?.!,])", r" ", text)

    def predict(self, question):
        question = self.clean_text(question)
        tokens = self.tokenizer.tokenize(question)
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)

        max_len = 40
        padded_ids = token_ids[:max_len] + [self.tokenizer.pad_token_id] * max(0, max_len - len(token_ids))
        mask = [1 if i < len(padded_ids) else 0 for i in range(max_len)]

        token_ids = torch.LongTensor(padded_ids).unsqueeze(0).to(self.device)
        mask = torch.LongTensor(mask).unsqueeze(0).to(self.device)

        with torch.no_grad():
            logits = self(input_ids=token_ids, attention_mask=mask)
            predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label


# 모델 인스턴스화 및 state_dict 로드
# model = QuestionClassification()
# model.load_state_dict(torch.load('/content/drive/MyDrive/Final_project/q_classifier/question_classification_model.pth', map_location=model.device))
# model.eval()

# 사용자로부터 질문 받기 및 예측 라벨 출력
# while True:
#     question = input("질문을 입력하세요: ")
#     if question.lower() == '종료':
#         break
#     predicted_label = model.predict(question)
#     print(f"예측된 라벨: {predicted_label}")


In [8]:
# 요리 추천 질문 응답 모델
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

class FoodSuggestion():
    def __init__(self):
        # 특수 토큰 정의
        self.Q_TKN = "<usr>"
        self.A_TKN = "<sys>"
        self.BOS = '</s>'
        self.EOS = '</s>'
        self.MASK = '<unused0>'
        self.SENT = '<unused1>'
        self.PAD = '<pad>'

        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2',
                                                                 bos_token=self.BOS,
                                                                 eos_token=self.EOS,
                                                                 unk_token="<unk>",
                                                                 pad_token=self.PAD,
                                                                 mask_token=self.MASK)
        self.model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        state_dict = torch.load('/content/drive/MyDrive/Final_project/food_suggestion/kogpt2_chatbot_model.pt', map_location=self.device)

        self.model.load_state_dict(state_dict)
        self.model.to(self.device)

    def predict(self, user_text):
        self.model.eval()
        with torch.no_grad():
          a = ''
          while True:
            input_ids = torch.LongTensor(self.tokenizer.encode(self.Q_TKN + user_text + self.SENT + self.A_TKN + a,
                                                               add_special_tokens=True)).unsqueeze(dim=0)
            input_ids = input_ids.to(self.device)
            outputs = self.model(input_ids)

            logits = outputs.logits

            pred = torch.argmax(logits, dim=-1)
            gen_token = self.tokenizer.convert_ids_to_tokens(pred[0].cpu().numpy().tolist())[-1]
            if gen_token == self.EOS:
              break
            a += gen_token.replace("▁", " ")

        return a.strip()


# 사용 예시 or 빠른 테스트
# model = FoodSuggestion()
# while True:
#   user_text = input('사용자: ')
#   if user_text == 'quit':
#     break
#   print('챗봇:', model.predict(user_text))


In [ ]:
class ModelManager:
    def __init__(self):
        # 모델 인스턴스를 저장할 딕셔너리
        self.models = {}

    def load_model(self, model_name, model_class, *args, **kwargs):
        """
        만든 모델들을 초기화하고 딕셔너리에 담기.
        args, kwargs 는 모델에 따라 필요하면 넣는 인자값들.
        현재 우리 프로젝트에서는 아직은 사용하는 모델들은 없음.
        """
        model = model_class(*args, **kwargs)
        if model_class == QuestionClassification:
            model.load_state_dict(torch.load('/content/drive/MyDrive/Final_project/q_classifier/question_classification_model_jw1.pth', map_location=model.device))
            model.eval()

        self.models[model_name] = model

    def get_model(self, model_name):
        """
        지정된 이름의 모델을 반환하는 메서드.
        model_name: 반환할 모델의 이름
        """
        return self.models.get(model_name)

    def predict(self, model_name, user_text ):
      """
      딕셔너리에 저장된 각 모델들 중에서 model_name에 해당되는 모델의 predic()메서드를 호출하는 메서드
      """

      return self.models[model_name].predict(user_text)



# 모델 메니저 클래스를 이용해서 모델들 사용해보기.


# 1.일단 사용할 모델들을 메니저에 load_model메서드를 이용해서 넣기.
model_manager = ModelManager()
model_manager.load_model('classifier', QuestionClassification) # 모델 이름은 그때그때 아무것나 식별하기 좋고 기억하기좋은걸로 해도됨.
model_manager.load_model("basedName", BasedNameModel)
model_manager.load_model("basedIngredient", BasedIngredientModel)
model_manager.load_model("classifyIngredients", ClassifierIngredients)
model_manager.load_model("foodSuggestion", FoodSuggestion)





# 2.나중에 예측시에는 이런식으로 사용
# user_text = '오늘은 돼지고기 많이 썰어 넣은 얼큰한 김치찌개가 먹고 싶어.'
# response_from_chatbot1 = model_manager.predict("basedName", user_text)
# print(response_from_chatbot1)
# response_from_chatbot2 = model_manager.predict("basedIngredient", user_text)
# print(response_from_chatbot2)


# CSV 파일을 열고 기록하기 위한 함수
def log_question_to_csv(question, predicted_label, response, file_path='/content/drive/MyDrive/Final_project/user_questions.csv'):
    # CSV 파일 열기 (없으면 새로 생성)
    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # 처음이라면, 컬럼 부터 정의
        if file.tell() == 0:
            writer.writerow(['Question', 'Predicted_Label', 'Response'])

        # 질문 기록
        writer.writerow([question, predicted_label, response])

def add_newlines(text, keywords):
    text = text.replace('Step 1', '일단다른거')

    for keyword in keywords:
        if keyword in ['Step', 'step', '조리 과정']:
            text = text.replace(keyword, '\n' + keyword)
        elif keyword in ['재료입니다:', '같습니다:']:
            text = text.replace(keyword, keyword + '\n')

    text = text.replace('일단다른거', 'Step 1')

    return text



# 챗봇으로 구현
# (0: 요리명 기반 질문, 1: 재료명 기반 질문, 2: 비요리 관련 질문들)
while True:
  question = input("사용자: ")
  if question in ['', ' ', '\n']:
    print(f'\033[33m'+'챗봇: 질문을 입력해주세요.'+'\033[0m')
    continue
  if question == '종료':
    break
  qeustion_label = model_manager.predict("classifier", question)
  if qeustion_label == 0:
    response_from_chat = model_manager.predict("basedName", question)

    # 혹시라도 추가적인 나중에 학습을 위해서 csv파일로 기록.
    log_question_to_csv(question, qeustion_label, response_from_chat)

    keywords = ['Step', 'step', '조리 과정', '같습니다:', '재료입니다:']
    result = add_newlines(response_from_chat,keywords)
    print(f'\033[33m챗봇: {result}\033[0m')

  elif qeustion_label == 1:
    response_from_chat = model_manager.predict("basedIngredient", question)

    # 혹시라도 추가적인 나중에 학습을 위해서 csv파일로 기록.
    log_question_to_csv(question, qeustion_label, response_from_chat)

    keywords = ['Step', 'step', '조리 과정', '같습니다:', '재료입니다:']
    result = add_newlines(response_from_chat,keywords)
    print(f'\033[33m챗봇: {result}\033[0m')

    # print(f'\033[31m' + '챗봇: 재료 기반 질문은 아직 미구현 상태입니다.' + '\033[0m')
  elif qeustion_label == 2:
    # 혹시라도 추가적인 나중에 학습을 위해서 csv파일로 기록.
    alert_msg = "챗봇: 요리명 기반, 혹은 재료명 기반으로 질문을 해주세요. 그 외 질문은 처리할 수 없습니다.."
    log_question_to_csv(question, qeustion_label, alert_msg)
    print(f'\033[33m'+ alert_msg +'\033[0m')


  elif qeustion_label == 3:
    response_from_chat = model_manager.predict("foodSuggestion", question)

    # 혹시라도 추가적인 나중에 학습을 위해서 csv파일로 기록
    log_question_to_csv(question, qeustion_label, response_from_chat)
    print(f'\033[33m'+ response_from_chat +'\033[0m')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


챗봇: 요리명 기반, 혹은 재료명 기반으로 질문을 해주세요. 그 외 질문은 처리할 수 없습니다..
챗봇: 모든 재료가 포함된 요리: 채식 순두부찌개
챗봇: 재료가 가장 많이 포함된 요리: 순두부찌개  굴순두부찌개
챗봇: 모든 재료가 포함된 요리: 두부두루치기  두부전  두부전
챗봇: 두부전을 만들기 위해 필요한 재료입니다:
 ['두부 1/2모'  '계란 1과1/2개'  '실파 2뿌리'  '카놀라유 약간'  '소금 약간'] 
조리 과정은 다음과 같습니다:
 ['Step 1: 두부는 이등분 해서 1cm 두께로 썰고 앞뒤로 소금을 조금씩 뿌려둔다 '  '
Step 2: 실파는 깨끗이 씻어 송송 썰어 준비한다 '  '
Step 3: 두부를 밀가루  계란물 순으로 옷을 입혀준다 '  '
Step 4: 프라이팬이 달궈지면 식용유를 두르고 3의 두부를 올리고 두부 중앙에 실파를 올려 앞뒤로 노릇하게 지진다 ']
챗봇: 재료가 가장 많이 포함된 요리: 순두부찌개  매운순대볶음
챗봇: 모든 재료가 포함된 요리: 순두부찌개  순두부찌개
챗봇: 재료가 가장 많이 포함된 요리: 순두부찌개  굴순두부찌개
챗봇: 모든 재료가 포함된 요리: 돼지고기겨자채보쌈
챗봇: 돼지고기오징어쌈밥을 만들기 위해 필요한 재료입니다:
 ['율무밥: 1/2공기'  '돼지고기(다진 것): 50g'  '오징어(손질한 것): 1/4마리'  '양배추잎: 1장'  '깻잎: 6장'  '포도씨유: 약간'  '고춧가루: 2작은술'  '고추장: 1작은술'  '청주: 1작은술'  '올리고당: 1작은술'  '다진 마늘: 1/2작은술'  '간장: 1/2작은술'  '후춧가루: 약간'] 
조리 과정은 다음과 같습니다:
 ['Step 1: 다진 돼지고기는 종이타월에 감싸 핏물을 제거하세요 '  '
Step 2: 양배추 잎은 잘게 다져주세요 '  '
Step 3: 오징어는 끓는 물에 살짝 데친 후 잘게 다져주세요 '  '
Step 4: 양념 재료를 볼에 모두 넣어 고루 섞어주세요 '  '
Step 5: 다진 돼지고기와 오징어  양배추